In [48]:
import pandas as pd
import pathlib
import numpy as np

## Load Data

In [54]:
raw_data_dir = pathlib.Path("../../data/raw/soroush_2")

trans_res_path = raw_data_dir.joinpath('2023.09.06 BSA_OVA_quant_super_akin.csv')
trans_res_df_raw = pd.read_csv(trans_res_path)
trans_res_df_raw.columns = trans_res_df_raw.columns.map(str.lower)
trans_res_df_raw.head()

,peptide,protein,replicate,precursor mz,precursor charge,product mz,product charge,fragment ion,retention time,area,background
0,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,844.42355,2,844.42355,2,precursor,47.00,7.575379e+09,133355208
1,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_SDC_W1730_Sup_D0_column12_2uL_80,844.42355,2,844.42355,2,precursor,45.96,9.467220e+09,98842
2,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_SDC_W1730_Sup_D1_column12_2uL_141,844.42355,2,844.42355,2,precursor,46.89,5.926985e+07,0
3,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_SDC_W1730_Sup_D1_column12_2uL_159,844.42355,2,844.42355,2,precursor,45.72,3.679658e+09,20224484
4,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_SDC_W1730_Sup_D2_column12_2uL_32,844.42355,2,844.42355,2,precursor,46.89,3.454373e+09,30604174


In [50]:

trans_res_df_raw.peptide.unique()

array(['GGLEPINFQTAADQAR', 'AEFVEVTK', 'YICDNQDTISSK'], dtype=object)

## Compute Area ratio

In [57]:
# def grp_area_ratio(grouped_df:pd.DataFrame):
    
#     """compute the ratio of areas with higer precursor mz value as numerator."""

#     df = grouped_df[['precursor mz', 'area']].copy()
#     df = df.sort_values(by='precursor mz', ascending=True)
#     area_ratio = df['area'].iloc[1] / df['area'].iloc[0]

#     return area_ratio

def grp_area_ratio_n_area_cols(grouped_df:pd.DataFrame):
    
    """compute the ratio of areas with higer precursor mz value as numerator."""

    df = grouped_df[['precursor mz', 'area']].copy()
    df = df.sort_values(by='precursor mz', ascending=True)
    area_ratio = df['area'].iloc[1] / df['area'].iloc[0]
    area_min, area_max  =  df['area'].iloc[0], df['area'].iloc[1]

    col_vals = {'area_ratio': area_ratio, 'area_min':area_min, 'area_max':area_max}

    return pd.Series(col_vals)

cols_to_group_by = ['peptide', 'protein', 'replicate', 'product charge', 'fragment ion']
# df_area_ratio = trans_res_df_raw.groupby(cols_to_group_by).apply(grp_area_ratio).reset_index(name='area_ratio')
df_area_ratio = trans_res_df_raw.groupby(cols_to_group_by).apply(grp_area_ratio_n_area_cols).reset_index()


# df_area_ratio = df_area_ratio[df_area_ratio['fragment ion'] == 'precursor']  # select only precursor
print(df_area_ratio.shape)
df_area_ratio.head()

c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in double_scalars
c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars


(1080, 8)


,peptide,protein,replicate,product charge,fragment ion,area_ratio,area_min,area_max
0,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,b2,0.000344,3.157382e+09,1085662.0
1,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y2,0.000128,4.009150e+08,51280.0
2,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y4,0.000017,6.640748e+08,11599.0
3,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y5,0.000003,8.254646e+08,2376.0
4,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y6,0.000171,2.778528e+09,475204.0


## Get peptide Dilution Conc.

In [58]:
df_peptide_dilution_conc = pd.read_csv(raw_data_dir.joinpath('Soroush_2023_09_06_peptide_dilution_conc.csv'))
# df_peptide_dilution_conc.columns


df_peptide_dilution_conc.head()
raw_dilution_col_names = df_peptide_dilution_conc.columns[1:]


df_peptide_dilution_conc = df_peptide_dilution_conc.melt(id_vars='Peptides', value_vars=raw_dilution_col_names, 
                                                        var_name='dilution', value_name='heavy_conc')
df_peptide_dilution_conc['dilution'] = df_peptide_dilution_conc['dilution'].apply(lambda x: x[:2])
df_peptide_dilution_conc['peptide_dilution_name'] = df_peptide_dilution_conc['Peptides'] + '_' + \
df_peptide_dilution_conc['dilution'] 
df_peptide_dilution_conc.head()

,Peptides,dilution,heavy_conc,peptide_dilution_name
0,GGLEPINFQTAADQAR,D1,0.736000,GGLEPINFQTAADQAR_D1
1,AEFVEVTK,D1,0.403667,AEFVEVTK_D1
2,YICDNQDTISSK,D1,1.257333,YICDNQDTISSK_D1
3,GGLEPINFQTAADQAR,D2,0.184000,GGLEPINFQTAADQAR_D2
4,AEFVEVTK,D2,0.100917,AEFVEVTK_D2


In [59]:



def get_peptide_dilution_comb(row):
    """function to create a new column from peptide and dilution name."""
    peptide = row['peptide']
    replicate  = row['replicate']
    dilution = get_dilution_from_replicate(rep_string=replicate)

    return peptide + '_' + dilution

df_peptide_dilution_conc = pd.read_csv(raw_data_dir.joinpath('Soroush_2023_09_06_peptide_dilution_conc.csv'))
raw_dilution_col_names = df_peptide_dilution_conc.columns[1:]

df_peptide_dilution_conc = df_peptide_dilution_conc.melt(id_vars='Peptides', value_vars=raw_dilution_col_names, 
                                                        var_name='dilution', value_name='heavy_conc')
df_peptide_dilution_conc['dilution'] = df_peptide_dilution_conc['dilution'].apply(lambda x: x[:2])
peptide_dilution_name_col = 'peptide_dilution_name'
df_peptide_dilution_conc[peptide_dilution_name_col] = df_peptide_dilution_conc['Peptides'] + '_' + \
                                                    df_peptide_dilution_conc['dilution'] 
df_peptide_dilution_conc.head()


,Peptides,dilution,heavy_conc,peptide_dilution_name
0,GGLEPINFQTAADQAR,D1,0.736000,GGLEPINFQTAADQAR_D1
1,AEFVEVTK,D1,0.403667,AEFVEVTK_D1
2,YICDNQDTISSK,D1,1.257333,YICDNQDTISSK_D1
3,GGLEPINFQTAADQAR,D2,0.184000,GGLEPINFQTAADQAR_D2
4,AEFVEVTK,D2,0.100917,AEFVEVTK_D2


### Merge data

In [60]:
def get_dilution_from_replicate(rep_string:str):
    """Get the dilution symbol from the replicate name."""

    str_to_check = rep_string.split('_')[4]
    if str_to_check.startswith('col'):
        dilution = 'D0'
    else:
        dilution = str_to_check

    return dilution

df_area_ratio[peptide_dilution_name_col] = df_area_ratio.apply(get_peptide_dilution_comb, axis=1)



df_area_ratio_conc = pd.merge(df_area_ratio, right=df_peptide_dilution_conc, how='left', 
                              on=peptide_dilution_name_col, suffixes=('', '_y'))
df_area_ratio_conc.drop(df_area_ratio_conc.filter(regex='_y$').columns, axis=1, inplace=True)
df_area_ratio_conc = df_area_ratio_conc.dropna(axis=0) # drop nan
# drop inf
df_area_ratio_conc.replace([np.inf, -np.inf], np.nan, inplace=True)
df_area_ratio_conc.dropna(subset=["area_ratio"], how="all", inplace=True)

# def get_rep_plot_cat(rep_val:str):
#     """Get the plot category (C18_SDS, C18_SDC, etc..)."""

#     select_elements = rep_val.split('_')[1:3]

#     cat = '_'.join(select_elements)
#     return cat

# TODO: - reverse x and y in plots: ratio:y, conc: x
#- TODO: further break replicate into 2 groups as shown in the excel, the greens are one group and the white is another.
def get_rep_plot_cat(row):
    """Get the plot category (C18_SDS, C18_SDC, etc..)."""

    replicate = row['replicate']
    peptide = row['peptide']
    fragment_ion = row['fragment ion']

    rep_elements = replicate.split('_')[0:3]

    cat_name = [peptide] + [fragment_ion] + rep_elements
    cat_name = '_'.join(cat_name)
    return cat_name

def get_order(grouped_df):

    rep_plot_cats = grouped_df['replicate'].values
    order_vals = []
    for rep_value in grouped_df['replicate'].values:
        rep_partial = rep_value.split('_')[:-1]
        rep_last_val = int(rep_value.split('_')[-1])

        for rep_value_2 in grouped_df['replicate'].values:
            rep_partial_2 = rep_value_2.split('_')[:-1]
            rep_last_val_2 = int(rep_value_2.split('_')[-1])

            found_match = False
            if rep_value != rep_value_2 and rep_partial == rep_partial_2:
                found_match = True
                if rep_last_val < rep_last_val_2:
                    order_val = 'low'
                elif rep_last_val > rep_last_val_2:
                    order_val = 'high'
                break
                
        if found_match:
            order_vals.append(order_val)
        else:
            order_vals.append('no match')

        
    
    grouped_df['order'] = order_vals
    return grouped_df
# create plot categories
df_area_ratio_conc['plot_cat'] = df_area_ratio_conc.apply(get_rep_plot_cat, axis=1)
order = df_area_ratio_conc.groupby('plot_cat').apply(get_order)['order']
df_area_ratio_conc['plot_cat'] = df_area_ratio_conc['plot_cat'] + '_' + order
df_area_ratio_conc.head()
# print(df_area_ratio_conc.isna().sum())
# df_area_ratio_conc = df_area_ratio_conc[df_area_ratio_conc['area_ratio'].notna()]
# print(df_area_ratio_conc.isna().sum())
df_area_ratio_conc.head()


,peptide,protein,replicate,product charge,fragment ion,area_ratio,area_min,area_max,peptide_dilution_name,Peptides,dilution,heavy_conc,plot_cat
0,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,b2,0.000344,3.157382e+09,1085662.0,AEFVEVTK_D0,AEFVEVTK,D0,0.0,AEFVEVTK_b2_Soroush_SDC_W1730_low
1,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y2,0.000128,4.009150e+08,51280.0,AEFVEVTK_D0,AEFVEVTK,D0,0.0,AEFVEVTK_y2_Soroush_SDC_W1730_low
2,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y4,0.000017,6.640748e+08,11599.0,AEFVEVTK_D0,AEFVEVTK,D0,0.0,AEFVEVTK_y4_Soroush_SDC_W1730_low
3,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y5,0.000003,8.254646e+08,2376.0,AEFVEVTK_D0,AEFVEVTK,D0,0.0,AEFVEVTK_y5_Soroush_SDC_W1730_low
4,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y6,0.000171,2.778528e+09,475204.0,AEFVEVTK_D0,AEFVEVTK,D0,0.0,AEFVEVTK_y6_Soroush_SDC_W1730_low


In [65]:
df_area_ratio_conc['plot_cat'][:20]

0                   AEFVEVTK_b2_Soroush_SDC_W1730_low
1                   AEFVEVTK_y2_Soroush_SDC_W1730_low
2                   AEFVEVTK_y4_Soroush_SDC_W1730_low
3                   AEFVEVTK_y5_Soroush_SDC_W1730_low
4                   AEFVEVTK_y6_Soroush_SDC_W1730_low
5            AEFVEVTK_precursor_Soroush_SDC_W1730_low
6      AEFVEVTK_precursor [M+1]_Soroush_SDC_W1730_low
7      AEFVEVTK_precursor [M+2]_Soroush_SDC_W1730_low
8                   AEFVEVTK_y6_Soroush_SDC_W1730_low
9                  AEFVEVTK_b2_Soroush_SDC_W1730_high
10                 AEFVEVTK_y2_Soroush_SDC_W1730_high
11                 AEFVEVTK_y4_Soroush_SDC_W1730_high
12                 AEFVEVTK_y5_Soroush_SDC_W1730_high
13                 AEFVEVTK_y6_Soroush_SDC_W1730_high
14          AEFVEVTK_precursor_Soroush_SDC_W1730_high
15    AEFVEVTK_precursor [M+1]_Soroush_SDC_W1730_high
16    AEFVEVTK_precursor [M+2]_Soroush_SDC_W1730_high
17                 AEFVEVTK_y6_Soroush_SDC_W1730_high
18                  AEFVEVTK

## Get Linear Fit parameters

In [61]:
from sklearn import linear_model, metrics
import matplotlib.pyplot as plt

def get_linear_fit(data:pd.DataFrame, plot_cat:str):

    cat_data = data[data['plot_cat'] == plot_cat]
    # print(cat_data)

    # fig, ax = plt.subplots()

    x = cat_data['area_ratio'].values
    y = cat_data['heavy_conc'].values

    # ax.scatter(x, y)

    x = x[:, np.newaxis]
    y = y[:, np.newaxis]

    model = linear_model.LinearRegression()

    model.fit(x, y)

    y_fit = model.predict(x)

    # ax.plot(x.flatten(), y_fit.flatten())

    r2 = metrics.r2_score(y, y_fit)
    intercept = model.intercept_.squeeze()
    grad = model.coef_[0].squeeze()


    # print(f"R^2: {r2}")
    # print(f"intercept: {intercept}")
    # print(f"gradient:{grad}")

    # plt.show()
    
    return (r2, intercept, grad)

plot_cats = df_area_ratio_conc['plot_cat'].unique()

# plot_df = df_area_ratio_conc[df_area_ratio_conc.peptide=='GGLEPINFQTAADQAR'] # get only one peptide
# plot_df = plot_df[[peptide_dilution_name_col, 'area_ratio', 'heavy_conc', 'plot_cat']]
plot_df = df_area_ratio_conc


cats = []
r2s = []
intercepts = []
grads = []

for cat in plot_cats:
    # print(cat)
    r2, intercept, grad = get_linear_fit(plot_df, cat)
    cats.append(cat)
    r2s.append(r2)
    intercepts.append(intercept)
    grads.append(grad)
    



df_plot_cat_fit_params = pd.DataFrame({'plot_cat':cats,
              'R2':r2s,
              'intercept':intercepts,
              'gradient':grads})




c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\sklearn\metrics\_regression.py:682: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\sklearn\metrics\_regression.py:682: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\sklearn\metrics\_regression.py:682: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\sklearn\metrics\_regression.py:682: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\sklearn\metrics\_regression.py:682: UndefinedMetricWarning: R^2 scor

In [62]:
df_area_ratio_conc_n_fit_params = pd.merge(df_area_ratio_conc,
                                           right=df_plot_cat_fit_params,
                                           on='plot_cat',
                                           how='left')

df_area_ratio_conc_n_fit_params
# save to csv file
# df_area_ratio_conc_n_fit_params.to_csv('area_ratio_conc_n_fit_params.csv', index=False)
    

,peptide,protein,replicate,product charge,fragment ion,area_ratio,area_min,area_max,peptide_dilution_name,Peptides,dilution,heavy_conc,plot_cat,R2,intercept,gradient
0,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,b2,0.000344,3.157382e+09,1085662.0,AEFVEVTK_D0,AEFVEVTK,D0,0.000000,AEFVEVTK_b2_Soroush_SDC_W1730_low,0.989631,0.012405890833013133,0.08596065728380857
1,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y2,0.000128,4.009150e+08,51280.0,AEFVEVTK_D0,AEFVEVTK,D0,0.000000,AEFVEVTK_y2_Soroush_SDC_W1730_low,0.988234,0.013288713001366886,0.08012130218963648
2,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y4,0.000017,6.640748e+08,11599.0,AEFVEVTK_D0,AEFVEVTK,D0,0.000000,AEFVEVTK_y4_Soroush_SDC_W1730_low,0.988140,0.013200529200107475,0.08030893120709683
3,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y5,0.000003,8.254646e+08,2376.0,AEFVEVTK_D0,AEFVEVTK,D0,0.000000,AEFVEVTK_y5_Soroush_SDC_W1730_low,0.988700,0.012942231199371629,0.08461953149401089
4,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y6,0.000171,2.778528e+09,475204.0,AEFVEVTK_D0,AEFVEVTK,D0,0.000000,AEFVEVTK_y6_Soroush_SDC_W1730_low,0.988829,0.012583549457748144,0.08106830937786105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_WW_Sup_D4_column12_2uL_89,1,y9,0.708577,2.510951e+07,17792034.0,YICDNQDTISSK_D4,YICDNQDTISSK,D4,0.007858,YICDNQDTISSK_y9_Soroush_SDC_WW_high,0.999989,0.0023662317061040605,0.0044031495977072885
1059,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_WW_Sup_D4_column12_2uL_89,2,precursor,0.729514,6.567330e+08,479095968.0,YICDNQDTISSK_D4,YICDNQDTISSK,D4,0.007858,YICDNQDTISSK_precursor_Soroush_SDC_WW_high,0.954948,0.0817255866068883,0.003969443021241678
1060,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_WW_Sup_D4_column12_2uL_89,2,precursor [M+1],0.689912,4.416241e+08,304681568.0,YICDNQDTISSK_D4,YICDNQDTISSK,D4,0.007858,YICDNQDTISSK_precursor [M+1]_Soroush_SDC_WW_high,0.974994,0.06357799706501588,0.0037899825463475773
1061,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_WW_Sup_D4_column12_2uL_89,2,precursor [M+2],0.637243,1.895787e+08,120807784.0,YICDNQDTISSK_D4,YICDNQDTISSK,D4,0.007858,YICDNQDTISSK_precursor [M+2]_Soroush_SDC_WW_high,0.956988,0.08098105477323397,0.0036560501234841063


# Fit metric

In [63]:
def get_fit_param_cat(row):
    """Get the fit param category (C18_SDS, C18_SDC, etc..)."""

    replicate = row['replicate']
    peptide = row['peptide']

    rep_elements = replicate.split('_')[0:3]

    cat_name = [peptide]  + rep_elements
    cat_name = '_'.join(cat_name)
    return cat_name

def qtest(data, right=True):

    sorted_data = sorted(data)

    if right:
        gap = sorted_data[-1] - sorted_data[-2]

    else:
        gap = sorted_data[1] - sorted_data[0]

    try:
        range = sorted_data[-1] - sorted_data[0]
        q_val = gap / range
    except ZeroDivisionError:
        q_val = gap

    return q_val

def get_grp_fit_agg(grouped_df:pd.DataFrame):
    """Get the aggregate values for each group."""

    df = grouped_df[['intercept', 'gradient', 'R2']].copy()
    gradient = df['gradient'].values
    intercept = df['intercept'].values
    r2 = df['R2'].values

    mean_grad = gradient.mean()
    stdv_grad = gradient.std()
    cov_grad = stdv_grad / mean_grad
    qtest_grad_right = qtest(gradient, right=True)

    mean_intercept = intercept.mean()
    stdv_intercept = intercept.std()
    cov_intercept = stdv_intercept / mean_intercept
    qtest_intercept_right = qtest(intercept, right=True)

    mean_r2 = r2.mean()
    stdv_r2 = r2.std()
    cov_r2 = stdv_r2 / mean_r2
    qtest_r2_right = qtest(r2, right=True)

    cols_dict = {'mean_grad':mean_grad, 'stdv_grad':stdv_grad,
                 'cov_grad':cov_grad, 'qtest_grad':qtest_grad_right, 
                 'mean_intercept':mean_intercept, 'stdv_intercept':stdv_intercept,
                 'cov_intercept':cov_intercept, 'qtest_intercept':qtest_intercept_right,
                 'mean_r2':mean_r2, 'stdv_r2':stdv_r2,
                 'cov_r2':cov_r2, 'qtest_r2':qtest_r2_right
                 }

    return pd.Series(cols_dict)


# create plot categories
temp = df_area_ratio_conc_n_fit_params.copy()
temp['fit_param_grp'] = temp.apply(get_fit_param_cat, axis=1)

cols_to_group_by = ['fit_param_grp']
df_fit_param_agg = temp.groupby(cols_to_group_by).apply(get_grp_fit_agg).reset_index()

df_w_fit_param_agg = pd.merge(left=temp, right=df_fit_param_agg, how='left', on=cols_to_group_by)
df_w_fit_param_agg.head(3)

,peptide,protein,replicate,product charge,fragment ion,area_ratio,area_min,area_max,peptide_dilution_name,Peptides,...,cov_grad,qtest_grad,mean_intercept,stdv_intercept,cov_intercept,qtest_intercept,mean_r2,stdv_r2,cov_r2,qtest_r2
0,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,b2,0.000344,3.157382e+09,1085662.0,AEFVEVTK_D0,AEFVEVTK,...,0.059483,0.0,0.014393,0.001572,0.109234,0.0,0.986283,0.002647,0.002684,0.0
1,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y2,0.000128,4.009150e+08,51280.0,AEFVEVTK_D0,AEFVEVTK,...,0.059483,0.0,0.014393,0.001572,0.109234,0.0,0.986283,0.002647,0.002684,0.0
2,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y4,0.000017,6.640748e+08,11599.0,AEFVEVTK_D0,AEFVEVTK,...,0.059483,0.0,0.014393,0.001572,0.109234,0.0,0.986283,0.002647,0.002684,0.0


In [64]:
result_data_folder = pathlib.Path("../../reports/ms_work")

df_w_fit_param_agg.to_csv(result_data_folder.joinpath('data_with_fit_param_aggregate_soroush_2.csv'), index=False)